In [129]:
import sys
sys.path.append("..")

import ccxt
import datetime as dt
import json
import numpy as np
import pandas as pd
import tqdm
from bitget.ws.bitget_ws_client import BitgetWsClient, SubscribeReq
from lib import utils

In [96]:
bitget = ccxt.bitget()

In [97]:
tickers = pd.DataFrame(bitget.fetchTickers())
markets = pd.DataFrame(bitget.fetchMarkets())

In [ ]:
ticker_info = pd.DataFrame.from_records(tickers.loc['info'].values)
ticker_info

In [ ]:
perps = markets[markets['swap']].set_index('id')
perp_info = ticker_info[ticker_info['symbol'].isin(perps.index)].sort_values('change24h', ascending=False)
perp_info

In [ ]:
perp_info.sort_values('usdtVolume')

In [110]:
top = perp_info['symbol'].values[:50]

all_symbols = perp_info['symbol'].values

# Screener

In [ ]:
for x in list(all_symbols):
    print(f"BITGET:{x}.P")

In [102]:
s = 'NEARUSDT'

In [ ]:
screen = []

changes = []
for s in tqdm.tqdm(all_symbols):
    df = pd.DataFrame(bitget.fetchOHLCV(symbol=perps.loc[s, 'symbol'], timeframe='5m', limit=600))
    df = df.set_index(pd.to_datetime(df[0].astype(int), unit='ms')).drop(columns=0).astype(float)
    df.columns=['open', 'high', 'low', 'close', 'volume']
    df.index.name = 'dt'
    df['volatility'] = 100*df['close'].pct_change().rolling(288).std()
    df['change'] = 100*df['close'].pct_change(288)
    df['change_peak'] = df['change'].abs().rolling(200).max()
    
    screen.append({
        'symbol': s,
        'volatility': df['volatility'].iloc[-1],
        'change': df['change'].iloc[-1],
        'change_peak': df['change_peak'].iloc[-1],
    })
    
    changes.append(df['change'].rename(s))
screen = pd.DataFrame(screen).set_index('symbol')
changes = pd.concat(changes, axis=1)

In [ ]:
changes.rank(axis=1).diff(72).iloc[-1].sort_values(ascending=False).head(20)

In [ ]:
changes.rank(axis=1).diff(72).iloc[-1].sort_values(ascending=False).tail(20)

In [ ]:
screen['change'].sort_values()

In [ ]:
screen[['volatility', 'change_peak']].rank(pct=True).mean(axis=1).sort_values(ascending=False)

In [ ]:
screen.sort_values('volatility', ascending=False)

# Streaming Bars

In [ ]:
wsclient = BitgetWsClient('wss://ws.bitget.com/v2/ws/public').build()

In [40]:
class BarStreamer():
    def __init__(self, wsclient, inst, interval):
        self.wsclient = wsclient
        self.inst = inst
        self.interval = interval
        self.channels = [SubscribeReq(instType="USDT-FUTURES", channel=f"candle{interval}", instId=inst)]
        self.data = None
        self.on_bar_close_callback = None
        return
    
    def subscribe(self):
        self.data = pd.DataFrame(bitget.fetchOHLCV(symbol=perps.loc[self.inst, 'symbol'], timeframe=self.interval, limit=1000))
        self.data = self.data.set_index(self.data[0].astype(int)).astype(float)
        self.wsclient.subscribe(self.channels, self.on_market_data)
        return
    
    def unsubscribe(self):
        self.wsclient.unsubscribe(self.channels)
        return
    
    def on_market_data(self, d):
        message = json.loads(d)
        new_df = pd.DataFrame(message['data'])
        new_df = new_df.set_index(new_df[0].astype(int)).astype(float)
        for i, row in new_df.iterrows():
            self.data.loc[i] = row
        if self.on_bar_close_callback: on_bar_close_callback(self.get_data())
    
    def get_data(self):
        return pd.DataFrame(self.data.values, pd.to_datetime(self.data.index, unit='ms'), ['ts', 'open', 'high', 'low', 'close', 'volume'])

In [41]:
bs = BarStreamer(wsclient, 'KEYUSDT', '1m')

In [ ]:
bs.subscribe()

In [43]:
# bs.unsubscribe()

In [84]:
swing_len = 5

def get_ob(df, swing_len):
    df['upper'] = df['high'].rolling(swing_len).max()
    df['lower'] = df['low'].rolling(swing_len).min()
    df['os'] = np.where(df['high'].shift(swing_len) > df['upper'], 0, np.where(df['low'].shift(swing_len) < df['lower'], 1, np.nan))
    df['os'] = df['os'].ffill()
    
    df['top_y'] = np.where((df['os'] == 0) & (df['os'].shift() != 0), df['high'].shift(swing_len), np.nan)
    df['top_x'] = np.where((df['os'] == 0) & (df['os'].shift() != 0), df.index.to_series().shift(swing_len),pd.NaT)
    df['btm_y'] = np.where((df['os'] == 1) & (df['os'].shift() != 1), df['low'].shift(swing_len), np.nan)
    df['btm_x'] = np.where((df['os'] == 1) & (df['os'].shift() != 1), df.index.to_series().shift(swing_len),pd.NaT)
    df['top_x'] = pd.to_datetime(df['top_x'])
    df['btm_x'] = pd.to_datetime(df['btm_x'])
    df['top_y'] = df['top_y'].ffill()
    df['top_x'] = df['top_x'].ffill()
    df['btm_y'] = df['btm_y'].ffill()
    df['btm_x'] = df['btm_x'].ffill()
    df['top_crossed'] = df.groupby((df['top_y'].dropna().diff() != 0).cumsum()).apply(lambda d: (d['close']>d['top_y']).cumsum().clip(0, 1)).droplevel('top_y')
    df['btm_crossed'] = df.groupby((df['btm_y'].dropna().diff() != 0).cumsum()).apply(lambda d: (d['close']<d['btm_y']).cumsum().clip(0, 1)).droplevel('btm_y')
    for i in df.index[utils.crossover(df['top_crossed'], 0.5)]:
        top_cross_data = df.loc[i]
        top_cross_df = df.loc[top_cross_data['top_x']:i].iloc[:-1]
        ts_bull_ob = top_cross_df['low'].idxmin()
        df.loc[i, 'bull_ob_top'] = top_cross_df.loc[ts_bull_ob, 'high']
        df.loc[i, 'bull_ob_btm'] = top_cross_df.loc[ts_bull_ob, 'low']
        df.loc[i, 'bull_ob_dt'] = ts_bull_ob
    for i in df.index[utils.crossover(df['btm_crossed'], 0.5)]:
        btm_cross_data = df.loc[i]
        btm_cross_df = df.loc[btm_cross_data['btm_x']:i].iloc[:-1]
        ts_bear_ob = btm_cross_df['high'].idxmax()
        df.loc[i, 'bear_ob_top'] = btm_cross_df.loc[ts_bear_ob, 'high']
        df.loc[i, 'bear_ob_btm'] = btm_cross_df.loc[ts_bear_ob, 'low']
        df.loc[i, 'bear_ob_dt'] = ts_bear_ob
    return df

In [89]:
df = get_ob(bs.get_data().iloc[:-1], 5)

In [94]:
le = df.groupby((~df['bear_ob_top'].isna()).cumsum()).apply(lambda d: utils.crossover(utils.crossover(d['close'], d['bear_ob_top'].ffill()).cumsum(), 0.5) ).droplevel('bear_ob_top')
se = df.groupby((~df['bull_ob_btm'].isna()).cumsum()).apply(lambda d: utils.crossover(utils.crossover(d['bull_ob_btm'].ffill(), d['close']).cumsum(), 0.5) ).droplevel('bull_ob_btm')

In [ ]:
se[se]